In [ ]:
# default_exp core.loss

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loss
> Different loss functions for the Temporal Model.

In [ ]:
#export
import numpy as np
import torch

### Mean per-joint position error
> Also known as the Euclidean distance:

Let $p$ be the predicted vector and $q$ be the ground truth for each joint, where $p,q \in \mathbb{R}^3$ then
$$ MPJPE(p,q) = \sqrt{(p_1 - q_1)^2 + (p_2 - q_2)^2 + (p_3 - q_3)^2}$$   

In [ ]:
#export
def mpjpe(predicted, target):
    """
    Mean per-joint position error (i.e mean Euclidean distance),
    often referred to as "Protocol #1" in many papers.
    """
    
    assert predicted.shape == target.shape
    return torch.mean(torch.norm(predicted - target, dim=len(target.shape)-1))

### Pose error: P-MPJPE
> Reports the error after alignment with the ground truth in translation, rotation, and scale.

In [ ]:
#export
def p_mpjpe(predicted, target):
    """
    Pose error: MPJPE after rigid alignment (scale, rotation, and translation),
    often referred to as "Protocol #2" in many papers.
    """

    assert predicted.shape == target.shape
    
    muX = np.mean(target, axis=1, keepdims=True)
    muY = np.mean(predicted, axis=1, keepdims=True)
    # Centering around the mean for the prediction and target vector. 
    # Calculates the L2-norm and divides the data with it. 
    X0 = target - muX
    Y0 = predicted - muY
    normX = np.sqrt(np.sum(X0**2, axis=(1, 2), keepdims=True))
    normY = np.sqrt(np.sum(Y0**2, axis=(1, 2), keepdims=True))
    X0 /= normX
    Y0 /= normY

    H = np.matmul(X0.transpose(0, 2, 1), Y0)
    U, s, Vt = np.linalg.svd(H)
    V = Vt.transpose(0, 2, 1)
    R = np.matmul(V, U.transpose(0, 2, 1))

    # Avoid improper rotations (reflections), i.e. rotations with det(R) = -1.
    sign_detR = np.sign(np.expand_dims(np.linalg.det(R), axis=1))
    V[:, :, -1] *= sign_detR
    s[:, -1] *= sign_detR.flatten()
    R = np.matmul(V, U.transpose(0, 2, 1)) # Rotation.

    tr = np.expand_dims(np.sum(s, axis=1, keepdims=True), axis=2)

    a = tr * normX / normY # Scale.
    t = muX - a*np.matmul(muY, R) # Translation.
    
    # Perform rigid transformation on the input.
    predicted_aligned = a*np.matmul(predicted, R) + t
    
    # Return MPJPE.
    return np.mean(np.linalg.norm(
        predicted_aligned - target, axis=len(target.shape)-1))


### Mean per joint velocity error
> Mean Euclidian distance of the 1:st derivative.

In [ ]:
#export
def mean_velocity_error(predicted, target):
    """
    Mean per-joint velocity error 
    (i.e. mean Euclidean distance of the 1st derivative).
    """
    assert predicted.shape == target.shape
    
    velocity_predicted = np.diff(predicted, axis=0)
    velocity_target = np.diff(target, axis=0)
    
    return np.mean(np.linalg.norm(
        velocity_predicted - velocity_target, axis=len(target.shape)-1))

In [ ]:
#export
def weighted_mpjpe(predicted, target, w):
    """
    Weighted mean per-joint position error (i.e. mean Euclidean distance)
    """
    assert predicted.shape == target.shape
    assert w.shape[0] == predicted.shape[0]
    return torch.mean(
        w * torch.norm(predicted - target, dim=len(target.shape)-1))

In [ ]:
#export
def n_mpjpe(predicted, target):
    """
    Normalized MPJPE (scale only), adapted from:
    https://github.com/hrhodin/UnsupervisedGeometryAwareRepresentationLearning/blob/master/losses/poses.py
    """
    assert predicted.shape == target.shape
    
    norm_predicted = torch.mean(torch.sum(
        predicted**2, dim=3, keepdim=True), dim=2, keepdim=True)
    norm_target = torch.mean(torch.sum(
        target*predicted, dim=3, keepdim=True), dim=2, keepdim=True)
    scale = norm_target / norm_predicted
    return mpjpe(scale * predicted, target)


In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_model.ipynb.
Converted 01_loss.ipynb.
Converted 02_skeleton.ipynb.
Converted 03_mocap_dataset.ipynb.
Converted 04_h36m_dataset.ipynb.
Converted 05_camera.ipynb.
Converted 06_quaternion.ipynb.
Converted 07_utils.ipynb.
Converted 08_generators.ipynb.
Converted 09_custom_dataset.ipynb.
Converted 10_visualization.ipynb.
Converted 11_arguments.ipynb.
Converted 12_data_utils.ipynb.
Converted 13_prepare_data_2d_custom.ipynb.
Converted 14_infer_video.ipynb.
Converted 15_prepare_data_COCO.ipynb.
Converted 16_pycococreatortools.ipynb.
Converted 17_format_qtmdata.ipynb.
Converted 18_runningpose_dataset.ipynb.
Converted 19_train_detectron2.ipynb.
Converted 20_transfer_model.ipynb.
Converted 21_prepare_data_3d.ipynb.
Converted index.ipynb.
